# Registros

Registros são o agrupamentos de campos correlacionados, criando uma única entidade com partes que a compõem.

Para programar com registros, foram implementadas classes para facilitar o agrupamento de campos e as gravações e leituras em arquivos.

A implementação do pacote [estrutarq](https://github.com/jandermoreira/estrutarq) está disponível no GitHub.

In [ ]:
!# Execute para baixar e atualizar o código disponível!
!git clone http://github.com/jandermoreira/estrutarq || \
  echo "Já foi executado antes? Basta baixar só uma vez..."
!cd estrutarq; git pull

## Registros brutos

Um registro bruto é apenas a concatenação de um ou mais campos, sem qualquer organização adicional para o registro.

In [ ]:
# Exemplos de registro bruto
from estrutarq.campo import *  # todos os campos
from estrutarq.registro import RegistroBruto

# Exemplo: aluno
registro_aluno = RegistroBruto(
    ("ra", CampoIntBinario()),
    ("nome", CampoCadeiaTerminador()),
    ("curso", CampoCadeiaPrefixado()),
    ("ingresso", CampoDataFixo())
)
registro_aluno.ra.valor = 123456
registro_aluno.nome.valor = "Neodímio Garcia"
registro_aluno.curso.valor = "Computacao"
registro_aluno.ingresso.valor = "2022-06-21"
print(registro_aluno)

Os bytes que compõem o campo bruto são apenas a justaposição dos bytes de seus campos.

Abaixo estão a representação padrão do Python para os bytes do registro seguido da lista de bytes em hexadecimal (ignore este comando esquisito; ele apenas mostra a sequência de bytes usando *artimanhas* da linguagem).

In [ ]:
print(registro_aluno.dado_formatado)
":".join([f"{um_byte:02X}" for um_byte in registro_aluno.dado_formatado])  # ignore

### Gravação

A gravação de um registro no arquivo é feita escrevendo-se seus bytes.

In [ ]:
# Gravação do registro
try:
  arquivo = open("alunos", "wb")
except IOError as excecao:
  print(f"Erro: {excecao}")
else:
  registro_aluno.escreva(arquivo)
  arquivo.close()

!hd alunos

### Leitura

A leitura de um campo bruto é feita pela leitura de cada campo individualmente. 

O conceito de registro fica estabelecido apenas pelo conhecimento dos campos e de suas organizações individuais.

In [ ]:
# Leitura do registro
novo_registro_aluno = RegistroBruto(
    ("ra", CampoIntBinario()),
    ("nome", CampoCadeiaTerminador()),
    ("curso", CampoCadeiaPrefixado()),
    ("ingresso", CampoDataFixo())
)

try:
  arquivo = open("alunos", "rb")
except IOError as excecao:
  print(f"Erro: {excecao}")
else:
  novo_registro_aluno.leia(arquivo)
  arquivo.close()

  print(novo_registro_aluno)

## Registros com terminadores

O registro pode ter um byte terminador, que tem que ser diferente do terminador de campo. Nesta implementação, o byte 0x01 é usado como terminador de registro.

In [ ]:
# Exemplos de registro com terminador
from estrutarq.registro import RegistroTerminador

# Exemplo: aluno
registro_aluno = RegistroTerminador(
    ("ra", CampoIntBinario()),
    ("nome", CampoCadeiaTerminador()),
    ("curso", CampoCadeiaPrefixado()),
    ("ingresso", CampoDataFixo())
)
registro_aluno.ra.valor = 123456
registro_aluno.nome.valor = "Neodímio Garcia"
registro_aluno.curso.valor = "Computacao"
registro_aluno.ingresso.valor = "2022-06-21"
print(registro_aluno)
print()
print(registro_aluno.dado_formatado)  # observe o escape de 0x01 no campo binário

## Registros prefixados pelo comprimento

O prefixo adotado para o comprimento é de dois bytes (16 bits), usando um número inteiro binário sem sinal. Desta forma, o comprimento máximo é $2^{16}$ bytes.

O comprimento é calculado sobre o comprimento dos campos já formatados.

In [ ]:
# Exemplos de registro com prefixo de comprimento
from estrutarq.registro import RegistroPrefixado

# Exemplo: aluno
registro_aluno = RegistroPrefixado(
    ("ra", CampoIntBinario()),
    ("nome", CampoCadeiaTerminador()),
    ("curso", CampoCadeiaPrefixado()),
    ("ingresso", CampoDataFixo())
)
registro_aluno.ra.valor = 123456
registro_aluno.nome.valor = "Neodímio Garcia"
registro_aluno.curso.valor = "Computacao"
registro_aluno.ingresso.valor = "2022-06-21"
print(registro_aluno)
print()
print(registro_aluno.dado_formatado)

## Registros de comprimento fixo

O registro assume, como um todo, comprimento fixo máximo.

O preenchimento dos registros é feito com o byte 0xFE.

In [ ]:
# Exemplos de registro com comprimento fixo
from estrutarq.registro import RegistroFixo

# Exemplo: aluno
registro_aluno = RegistroFixo(
    60,  # comprimento final em bytes
    ("ra", CampoIntBinario()),
    ("nome", CampoCadeiaTerminador()),
    ("curso", CampoCadeiaPrefixado()),
    ("ingresso", CampoDataFixo())
)
registro_aluno.ra.valor = 123456
registro_aluno.nome.valor = "Neodímio Garcia"
registro_aluno.curso.valor = "Computacao"
registro_aluno.ingresso.valor = "2022-06-21"
print(registro_aluno)
print()
print(registro_aluno.dado_formatado)

# Arquivos com registros

Um arquivo de registros pode, em um primeiro momento, utilizar uma sequência de registros, gravados um após o outro.

Segue, assim, um exemplo de criação de um arquivo para guardar várias registros.

Cada registro corresponde a um nome de estado da federação e sua respectiva sigla.

In [ ]:
# Escrita da lista em um arquivo
estados_e_siglas = [
  ("Acre", "AC"),
  ("Alagoas", "AL"),
  ("Amapá", "AP"),
  ("Amazonas", "AM"),
  ("Bahia", "BA"),
  ("Ceará", "CE"),
  ("Distrito Federal", "DF"),  # ok, ok, não é estado!
  ("Espírito Santo	", "ES"),
  ("Goiás", "GO"),
  ("Maranhão", "MA"),
  ("Mato Grosso", "MT"),
  ("Mato Grosso do Sul", "MS"),
  ("Minas Gerais", "MG"),
  ("Pará", "PA"),
  ("Paraíba", "PB"),
  ("Paraná", "PR"),
  ("Pernambuco", "PE"),
  ("Piauí", "PI"),
  ("Rio de Janeiro", "RJ"),
  ("Rio Grande do Norte", "RN"),
  ("Rio Grande do Sul", "RS"),
  ("Rondônia", "RO"),
  ("Roraima", "RR"),
  ("Santa Catarina", "SC"),
  ("São Paulo", "SP"),
  ("Sergipe", "SE"),
  ("Tocantins", "TO")
]

try:
  arquivo = open("estados", "wb")
except IOError as excecao:
  print(f"Erro: {excecao}")
else:
  info_estado = RegistroPrefixado(
      ("nome", CampoCadeiaTerminador()),
      ("sigla", CampoCadeiaFixo(2))
  )
  for nome, sigla in estados_e_siglas:
    info_estado.nome.valor = nome
    info_estado.sigla.valor = sigla
    info_estado.escreva(arquivo)
    print(f"{sigla} ", end = "")

  print
  arquivo.close()

In [ ]:
!hd estados

Finalmente, a recuperação da matriz a partir do arquivo

In [ ]:
# Leitura 
try:
  arquivo = open("estados", "rb")
except IOError as excecao:
  print(f"Erro: {excecao}")
else:
  info_estado = RegistroPrefixado(
    ("nome", CampoCadeiaTerminador()),
    ("sigla", CampoCadeiaFixo(2))
  )

  fim_do_arquivo = False
  while not fim_do_arquivo:
    try:
      info_estado.leia(arquivo)
    except EOFError:
      fim_do_arquivo = True
    else:
      print(f"{info_estado.nome} tem sigla {info_estado.sigla}")
  arquivo.close()